In [1]:
# !wget https://github.com/aisingapore/seacorenlp-data/raw/main/id/constituency/train.txt
# !wget https://github.com/aisingapore/seacorenlp-data/raw/main/id/constituency/test.txt

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
import numpy as np
import torch
import torch.nn as nn
from malaya.function.constituency import evaluate, trees_newline as trees
from transformers import AutoTokenizer, T5Config
from malaya.torch_model.t5 import T5Constituency
from tqdm import tqdm

2023-09-21 10:11:01.212774: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-21 10:11:01.291210: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-09-21 10:11:01.733338: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-09-21 10:11:01.733370: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [4]:
BERT_TOKEN_MAPPING = {
    "-LRB-": "(",
    "-RRB-": ")",
    "-LCB-": "{",
    "-RCB-": "}",
    "-LSB-": "[",
    "-RSB-": "]",
    "``": '"',
    "''": '"',
    "`": "'",
    '«': '"',
    '»': '"',
    '‘': "'",
    '’': "'",
    '“': '"',
    '”': '"',
    '„': '"',
    '‹': "'",
    '›': "'",
    "\u2013": "--", # en dash
    "\u2014": "--", # em dash
    }

def process_word(word):
    word = word.replace('\\/', '/').replace('\\*', '*')
    # Mid-token punctuation occurs in biomedical text
    word = word.replace('-LSB-', '[').replace('-RSB-', ']')
    word = word.replace('-LRB-', '(').replace('-RRB-', ')')
    if word == "n't" and cleaned_words:
        cleaned_words[-1] = cleaned_words[-1] + "n"
        word = "'t"
    return word

In [5]:
import collections

class Vocabulary(object):
    def __init__(self):
        self.frozen = False
        self.values = []
        self.indices = {}
        self.counts = collections.defaultdict(int)

    @property
    def size(self):
        return len(self.values)

    def value(self, index):
        assert 0 <= index < len(self.values)
        return self.values[index]

    def index(self, value):
        if not self.frozen:
            self.counts[value] += 1

        if value in self.indices:
            return self.indices[value]

        elif not self.frozen:
            self.values.append(value)
            self.indices[value] = len(self.values) - 1
            return self.indices[value]

        else:
            raise ValueError("Unknown value: {}".format(value))

    def index_or_unk(self, value, unk_value):
        assert self.frozen
        if value in self.indices:
            return self.indices[value]
        else:
            return self.indices[unk_value]

    def count(self, value):
        return self.counts[value]

    def freeze(self):
        self.frozen = True


In [6]:
train_treebank = trees.load_trees('train.txt')
train_parse = [tree.convert() for tree in train_treebank]

In [7]:
dev_treebank = trees.load_trees('test.txt')

In [8]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/nanot5-base-malaysian-cased')

In [9]:
label_vocab = Vocabulary()
label_vocab.index(())

tag_vocab = Vocabulary()
START = '<s>'
STOP = '</s>'
UNK = tokenizer.unk_token
TAG_UNK = "UNK"
tag_vocab.index(START)
tag_vocab.index(STOP)
tag_vocab.index(TAG_UNK)

for tree in train_parse:
    nodes = [tree]
    while nodes:
        node = nodes.pop()
        if isinstance(node, trees.InternalParseNode):
            label_vocab.index(node.label)
            nodes.extend(reversed(node.children))
        else:
            tag_vocab.index(node.tag)
            
tag_vocab.freeze()
label_vocab.freeze()

In [10]:
config = T5Config.from_pretrained('mesolitica/nanot5-small-malaysian-cased')
config.num_labels = label_vocab.size
config.num_tags = tag_vocab.size
config.tag_loss_scale = 1.0
config.label_vocab = {str(k): v for k, v in label_vocab.indices.items()}
config.tag_vocab = tag_vocab.indices

In [11]:
model = T5Constituency.from_pretrained('mesolitica/nanot5-small-malaysian-cased', config = config)
_ = model.cuda()

Some weights of T5Constituency were not initialized from the model checkpoint at mesolitica/nanot5-small-malaysian-cased and are newly initialized: ['encoder_encoder.attn_6.w_vs2', 'encoder_encoder.ff_7.w_2c.bias', 'encoder_encoder.attn_4.w_ks1', 'f_tag.1.a_2', 'encoder_encoder.ff_6.layer_norm.a_2', 'encoder_encoder.attn_0.layer_norm.a_2', 'encoder_encoder.attn_6.layer_norm.b_2', 'f_tag.3.weight', 'encoder_encoder.ff_0.w_2p.bias', 'encoder_encoder.ff_5.w_1p.weight', 'encoder_encoder.attn_2.w_vs1', 'encoder_encoder.attn_3.w_ks2', 'encoder_encoder.ff_1.w_2c.weight', 'encoder_encoder.ff_4.layer_norm.b_2', 'encoder_encoder.attn_3.proj1.weight', 'encoder_encoder.ff_6.w_1c.weight', 'f_tag.0.bias', 'f_label.1.a_2', 'f_tag.1.b_2', 'encoder_encoder.ff_2.layer_norm.b_2', 'encoder_encoder.attn_4.layer_norm.a_2', 'encoder_encoder.ff_4.w_1c.bias', 'encoder_encoder.attn_5.w_ks2', 'f_tag.3.bias', 'encoder_encoder.attn_1.proj1.weight', 'encoder_encoder.attn_4.proj1.weight', 'encoder_encoder.attn_7.lay

In [12]:
trainable_parameters = [param for param in model.parameters() if param.requires_grad]

In [13]:
trainer = torch.optim.AdamW(trainable_parameters, lr = 2e-4)

In [14]:
class BatchIndices:
    def __init__(self, batch_idxs_np):
        self.batch_idxs_np = batch_idxs_np
        self.batch_idxs_torch = torch.from_numpy(batch_idxs_np)
        self.batch_size = int(1 + np.max(batch_idxs_np))

        batch_idxs_np_extra = np.concatenate([[-1], batch_idxs_np, [-1]])
        self.boundaries_np = np.nonzero(batch_idxs_np_extra[1:] != batch_idxs_np_extra[:-1])[0]
        self.seq_lens_np = self.boundaries_np[1:] - self.boundaries_np[:-1]
        assert len(self.seq_lens_np) == self.batch_size
        self.max_len = int(np.max(self.boundaries_np[1:] - self.boundaries_np[:-1]))

In [15]:
def split_batch(sentences, golds, subbatch_max_tokens=3000):
    lens = [
        len(tokenizer.tokenize(' '.join([word for (_, word) in sentence]))) + 2
        for sentence in sentences
    ]

    lens = np.asarray(lens, dtype=int)
    lens_argsort = np.argsort(lens).tolist()

    num_subbatches = 0
    subbatch_size = 1
    while lens_argsort:
        if (subbatch_size == len(lens_argsort)) or (subbatch_size * lens[lens_argsort[subbatch_size]] > subbatch_max_tokens):
            yield [sentences[i] for i in lens_argsort[:subbatch_size]], [golds[i] for i in lens_argsort[:subbatch_size]]
            lens_argsort = lens_argsort[subbatch_size:]
            num_subbatches += 1
            subbatch_size = 1
        else:
            subbatch_size += 1
            
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(
            sentence + [pad_int] * (max_sentence_len - len(sentence))
        )
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [16]:
def process(sentences, golds = None):
    
    all_input_ids = []
    all_word_start_mask = []
    all_word_end_mask = []

    for snum, sentence in enumerate(sentences):

        tokens = []
        word_start_mask = []
        word_end_mask = []
        tokens.append(START)
        word_start_mask.append(1)
        word_end_mask.append(1)

        cleaned_words = []
        for _, word in sentence:
            cleaned_words.append(process_word(word))

        for word in cleaned_words:
            word_tokens = tokenizer.tokenize(word)
            for _ in range(len(word_tokens)):
                word_start_mask.append(0)
                word_end_mask.append(0)
            word_start_mask[len(tokens)] = 1
            word_end_mask[-1] = 1
            tokens.extend(word_tokens)
        tokens.append(STOP)
        word_start_mask.append(1)
        word_end_mask.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        all_input_ids.append(input_ids)
        all_word_start_mask.append(word_start_mask)
        all_word_end_mask.append(word_end_mask)
    
    padded = tokenizer.pad({
        'input_ids': all_input_ids,
    }, return_tensors = 'pt')

    all_word_start_mask = torch.from_numpy(np.array(pad_sentence_batch(all_word_start_mask, 0)[0]))
    all_word_end_mask = torch.from_numpy(np.array(pad_sentence_batch(all_word_end_mask, 0)[0]))
    
    padded['sentences'] = sentences
    padded['all_word_start_mask'] = all_word_start_mask
    padded['all_word_end_mask'] = all_word_end_mask
    
    packed_len = sum([(len(sentence) + 2) for sentence in sentences])
    i = 0
    tag_idxs = np.zeros(packed_len, dtype=int)
    batch_idxs = np.zeros(packed_len, dtype=int)
    for snum, sentence in enumerate(sentences):
        for (tag, word) in [(START, START)] + sentence + [(STOP, STOP)]:
            if golds is not None:
                tag_idxs[i] = tag_vocab.index_or_unk(tag, TAG_UNK)
            else:
                tag_idxs[i] = 0
            batch_idxs[i] = snum
            i += 1
    
    batch_idxs = BatchIndices(batch_idxs)
    padded['batch_idxs'] = batch_idxs
    tag_idxs = torch.from_numpy(tag_idxs)
    padded['tag_idxs'] = tag_idxs
    
    if golds is not None:
        gold_tag_idxs = tag_idxs
        padded['gold_tag_idxs'] = gold_tag_idxs
        padded['golds'] = golds
        
        
    if torch.cuda.is_available():
        for k in padded.keys():
            if isinstance(padded[k], torch.Tensor):
                padded[k] = padded[k].cuda()
    
        padded['batch_idxs'].batch_idxs_torch = padded['batch_idxs'].batch_idxs_torch.cuda()
    
    return padded

In [17]:
start_index = 0
batch_size = 4
batch_trees = train_parse[start_index:start_index + batch_size]
batch_sentences = [[(leaf.tag, leaf.word) for leaf in tree.leaves()] for tree in batch_trees]
for subbatch_sentences, subbatch_trees in split_batch(batch_sentences, batch_trees):
    print(model(**process(subbatch_sentences, subbatch_trees)))
    break

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


(tensor(256.2531, device='cuda:0', grad_fn=<AddBackward0>), tensor(288.7574, device='cuda:0', grad_fn=<MulBackward0>))


In [18]:
# model(**process(subbatch_sentences)), model(**process(subbatch_sentences, subbatch_trees))

In [19]:
batch_size = 16
epoch = 100

best_dev_fscore = -np.inf
patient = 10
current_patient = 0

for e in range(epoch):
    pbar = tqdm(range(0, len(train_parse), batch_size))
    losses = []
    for start_index in pbar:
        trainer.zero_grad()
        batch_loss_value = 0.0
        batch_trees = train_parse[start_index:start_index + batch_size]
        batch_sentences = [[(leaf.tag, leaf.word) for leaf in tree.leaves()] for tree in batch_trees]
        batch_num_tokens = sum(len(sentence) for sentence in batch_sentences)

        for subbatch_sentences, subbatch_trees in split_batch(batch_sentences, batch_trees):
            loss, tag_loss =  model(**process(subbatch_sentences, subbatch_trees))
            loss = tag_loss / len(subbatch_sentences) + loss / batch_num_tokens
            loss_value = float(loss.data.cpu().numpy())
            batch_loss_value += loss_value
            if loss_value > 0:
                loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(trainable_parameters, 5.0)
        trainer.step()
        losses.append(batch_loss_value)
        
    dev_predicted = []
    for dev_start_index in range(0, len(dev_treebank), batch_size):
        subbatch_trees = dev_treebank[dev_start_index:dev_start_index+batch_size]
        subbatch_sentences = [[(leaf.tag, leaf.word) for leaf in tree.leaves()] for tree in subbatch_trees]
        predicted, _ =  model(**process(subbatch_sentences))
        dev_predicted.extend([p.convert() for p in predicted])
    
    dev_fscore = evaluate.evalb('deprecated/EVALB', dev_treebank, dev_predicted)
    
    print(f'epoch: {e}, loss: {np.mean(losses)}, dev_fscore: {dev_fscore}')
    
    if dev_fscore.fscore > best_dev_fscore:
        best_dev_fscore = dev_fscore.fscore
        current_patient = 0
        model.save_pretrained('small')
    else:
        current_patient += 1
    
    if current_patient >= patient:
        break

100%|█████████████████████████████████████████| 500/500 [00:52<00:00,  9.45it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
epoch: 0, loss: 36.82793775939941, dev_fscore: (Recall=17.01, Precision=46.06, FScore=24.84, CompleteMatch=0.00, TaggingAccuracy=61.28)


100%|█████████████████████████████████████████| 500/500 [00:50<00:00,  9.82it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
epoch: 1, loss: 20.886111581802368, dev_fscore: (Recall=37.44, Precision=54.97, FScore=44.54, CompleteMatch=1.30, TaggingAccuracy=71.97)


100%|█████████████████████████████████████████| 500/500 [00:50<00:00,  9.85it/s]


...

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
epoch: 97, loss: 0.29544836299930466, dev_fscore: (Recall=76.96, Precision=80.66, FScore=78.76, CompleteMatch=19.00, TaggingAccuracy=93.99)


100%|█████████████████████████████████████████| 500/500 [00:49<00:00, 10.13it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
epoch: 98, loss: 0.2811927164471708, dev_fscore: (Recall=79.10, Precision=80.63, FScore=79.86, CompleteMatch=20.10, TaggingAccuracy=93.96)


100%|█████████████████████████████████████████| 500/500 [00:49<00:00, 10.07it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
epoch: 99, loss: 0.2393738731777994, dev_fscore: (Recall=78.56, Precision=80.29, FScore=79.41, CompleteMatch=20.20, TaggingAccuracy=94.09)


In [20]:
model_ = T5Constituency.from_pretrained('./small')

In [21]:
model_.push_to_hub('mesolitica/constituency-parsing-nanot5-small-malaysian-cased', safe_serialization = True)

model.safetensors:   0%|          | 0.00/273M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mesolitica/constituency-parsing-nanot5-small-malaysian-cased/commit/15f85af31523aed2eac7d3fedd5e1dcc981448c3', commit_message='Upload T5Constituency', commit_description='', oid='15f85af31523aed2eac7d3fedd5e1dcc981448c3', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
tokenizer.push_to_hub('mesolitica/constituency-parsing-nanot5-small-malaysian-cased', safe_serialization = True)

CommitInfo(commit_url='https://huggingface.co/mesolitica/constituency-parsing-nanot5-small-malaysian-cased/commit/be911c3f2acfa342a9f95ffed3c3d224435e33a4', commit_message='Upload tokenizer', commit_description='', oid='be911c3f2acfa342a9f95ffed3c3d224435e33a4', pr_url=None, pr_revision=None, pr_num=None)